Aknowledgements:
Some of the following code has been taken from the the following github repositiory, given in the main page of the shared task 8 and can be seen through this link: 
https://github.com/jorses/databench_eval/blob/main/examples/competition_baseline.py



In [1]:

%pip install databench-eval
%pip install pandas
%pip install datasets
%pip install tqdm
%pip install dotenv
%pip install rich
%pip install openai
%pip install --upgrade openai 


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note

In [2]:

import openai
import pandas as pd
import numpy as np
import subprocess
import shlex
import zipfile
from rich import print

from datasets import Dataset
from databench_eval import Runner, Evaluator

/Users/kurtik/Union College/WINTER 25/CSC306/CSC-306-Proj4/csc306.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the main QA data from three .txt files into a DataFrame
qa_df = pd.DataFrame()

# Read the answers from the .txt files into separate lists
with open("answers/answers.txt", "r") as f:
    answers = f.read().splitlines()

with open("answers/answers_lite.txt", "r") as f:
    sample_answers = f.read().splitlines()

with open("answers/semantics.txt", "r") as f:
    semantics = f.read().splitlines()

# Combine the lists into a DataFrame

# Load the dataset column from the specified file
qa_df = pd.read_csv(
    "competition/test_qa.csv" # file provided in the competition
)
qa_df["answer"] = answers
qa_df["sample_answer"] = sample_answers
qa_df["type"] = semantics


# Convert to Dataset
qa = Dataset.from_pandas(qa_df.head(100))
evaluator = Evaluator(qa=qa)

# print(answers)
# print(sample_answers)
# print(semantics[:5])

In [4]:
from zero_shot_incontext_learning import ZeroShotModel
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")




In [5]:
# Create model instance using the API key set in the environment variable
zero_shot = ZeroShotModel(api_key=api_key)
    
# Ask a question about a dataset
dataset_name = "071_COL"
question = "What is the most expensive city in this dataset?"

response = zero_shot.ask_question(dataset_name, question)
print(response)

{
    "answer": "Switzerland",
    "columns_used": ["Country", "Cost of Living Index"],
    "explanation": "The most expensive city is determined by the highest Cost of Living Index, which is Switzerland
with a value of 101.1."
}

In [ ]:

# Read the CSV file
qa_df = pd.read_csv("competition/test_qa.csv")

# Initialize an empty list to store the arrays
qa_list = []

# Iterate through each row in the DataFrame
for index, row in qa_df.iterrows():
    # Create an array with the question and respective file
    qa_array = [row['question'], row['dataset']]
    # Append the array to the list
    qa_list.append(qa_array)

print(qa_list)

In [ ]:
from tqdm import tqdm
import json


def get_predictions(qa_list):
    y_pred = []
    for row in tqdm(qa_list[:10], desc="Processing"):
        question = row[0]
        dataset_name = row[1]
        response = zero_shot.ask_question(dataset_name, question)
        response_json = json.loads(response)
        y_pred.append(response_json["answer"])
    return y_pred

y_pred = get_predictions(qa_list)



In [ ]:
def normalize_answers(y_pred):
    for i in range(len(y_pred)):
        if y_pred[i] == "Yes":
            y_pred[i] = "True"
        elif y_pred[i] == "No":
            y_pred[i] = "False"
    return y_pred
y_pred = normalize_answers(y_pred)
# Print the predictions
print("Predictions:")
print(y_pred)

In [ ]:
# Print the predictions
print("Predictions:")
print(y_pred)
# Print the sample answers
print("Sample Answers:")
print(sample_answers[:10])

In [ ]:
from evaluation import evaluate

evaluate(y_pred, sample_answers[:10])